# Machine Learning with PyTorch and Scikit-Learn  
# -- Code Examples

## Package version checks

Add folder to path in order to load from the check_packages.py script:

In [1]:
import sys
sys.path.insert(0, '..')

理解了！你想在你的Python環境中導入上級目錄（即父目錄）的模組或程式庫。這在處理專案結構或需要訪問其他目錄中的模組時很有用。

通過將上級目錄插入到`sys.path`中，可以讓Python解釋器找到並載入該目錄中的模組。這在開發和測試時尤其有用，可以避免將所有模組都放在同一目錄下，從而更好地組織代碼。

如果你有任何具體的問題或需要進一步的說明，請隨時告訴我！

Check recommended package versions:

In [2]:
from python_environment_check import check_packages


d = {
    'pandas': '1.3.2',
    'torch': '1.9.0',
    'torchtext': '0.11.0',
    'datasets': '1.11.0',
    'transformers': '4.9.1',
}
check_packages(d)

[OK] Your Python version is 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]
[OK] pandas 1.3.5
[OK] torch 1.10.0
[OK] torchtext 0.11.0
[OK] datasets 1.11.0
[OK] transformers 4.9.1


如果你想確認這些套件是否安裝在你的Python環境中，可以透過以下步驟來檢查：

1. **使用pip查看套件版本**：在終端或命令提示字元中執行以下命令，可以確認套件是否安裝以及其版本：

   ```bash
   pip show pandas torch torchtext datasets transformers
   ```

   這將顯示每個套件的詳細資訊，包括版本號碼。

2. **在Python中執行**：你也可以在Python中執行以下程式碼來確認這些套件的版本：

   ```python
   import pandas
   import torch
   import torchtext
   import datasets
   import transformers

   print("pandas 版本:", pandas.__version__)
   print("torch 版本:", torch.__version__)
   print("torchtext 版本:", torchtext.__version__)
   print("datasets 版本:", datasets.__version__)
   print("transformers 版本:", transformers.__version__)
   ```

   這將列印出每個套件的版本資訊。

如果你需要進一步的幫助或有其他問題，請隨時告訴我！

# Chapter 16: Transformers – Improving Natural Language Processing with Attention Mechanisms (Part 3/3)

**Outline**

- [Fine-tuning a BERT model in PyTorch](#Fine-tuning-a-BERT-model-in-PyTorch)
  - [Loading the IMDb movie review dataset](#Loading-the-IMDb-movie-review-dataset)
  - [Tokenizing the dataset](#Tokenizing-the-dataset)
  - [Loading and fine-tuning a pre-trained BERT model](#[Loading-and-fine-tuning-a-pre-trained-BERT-model)
  - [Fine-tuning a transformer more conveniently using the Trainer API](#Fine-tuning-a-transformer-more-conveniently-using-the-Trainer-API)
- [Summary](#Summary)

---

Quote from https://huggingface.co/transformers/custom_datasets.html:

> DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased , runs 60% faster while preserving over 95% of BERT's performances as measured on the GLUE language understanding benchmark.

---

In [ ]:
from IPython.display import Image

這行程式碼是在從IPython.display模組中導入Image類別。這個類別用於在IPython環境中顯示圖像。通常，你可以使用這個Image類別來載入並顯示本地檔案系統中的圖片，或者顯示從網路上獲取的圖片。

## Fine-tuning a BERT model in PyTorch

### Loading the IMDb movie review dataset


In [2]:
import gzip
import shutil
import time

import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext

import transformers
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification

讓我逐行解釋這些import語句的含意：

1. `import gzip`: 導入Python的gzip模組，用於處理gzip壓縮文件。
   
2. `import shutil`: 導入Python的shutil模組，用於高級文件操作，例如拷貝、刪除、移動文件和目錄。

3. `import time`: 導入Python的time模組，提供時間操作的函數，例如sleep()。

4. `import pandas as pd`: 導入Pandas庫，並使用`pd`作為別名。Pandas是用於數據處理和分析的Python庫，提供了強大的數據結構和工具。

5. `import requests`: 導入Python的requests庫，用於向網絡服務器發送HTTP請求和處理響應。

6. `import torch`: 導入PyTorch庫，用於深度學習和科學計算。

7. `import torch.nn.functional as F`: 導入PyTorch的nn.functional模組並使用F作為別名。這個模組包含了各種用於神經網絡操作的函數，如激活函數、損失函數等。

8. `import torchtext`: 導入PyTorch的torchtext庫，用於自然語言處理中文本數據的加載、處理和預處理。

9. `import transformers`: 導入transformers庫，這是Hugging Face開發的一個庫，用於自然語言處理中的預訓練模型、tokenization和fine-tuning等任務。

10. `from transformers import DistilBertTokenizerFast`: 從transformers庫中導入DistilBertTokenizerFast類別。這是一個快速的DistilBERT模型的分詞器，用於將文本轉換為模型可以處理的格式。

11. `from transformers import DistilBertForSequenceClassification`: 從transformers庫中導入DistilBertForSequenceClassification類別。這是一個DistilBERT模型的分類器，用於處理文本序列分類任務，如情感分析或文本分類。

這些import語句構建了一個Python環境，使得你可以在項目中使用各種庫和工具來進行數據處理、深度學習建模以及自然語言處理任務。

**General Settings**

In [3]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NUM_EPOCHS = 3

這段代碼設置了一些用於深度學習模型訓練的基本配置。讓我逐行解釋：

1. `torch.backends.cudnn.deterministic = True`: 這行代碼設置了CuDNN的deterministic模式為True。CuDNN是NVIDIA針對深度神經網絡的庫，deterministic模式確保在相同的輸入下，每次運行的結果都是一致的，這對於結果的可重現性很重要。

2. `RANDOM_SEED = 123`: 設置了一個隨機種子為123。這個隨機種子將用於各種隨機初始化，如PyTorch中的隨機初始化、數據加載時的隨機分割等，確保每次運行的結果都是一致的。

3. `torch.manual_seed(RANDOM_SEED)`: 使用上述隨機種子設置PyTorch的隨機種子，確保在可重現的情況下使用同一組隨機數。

4. `DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')`: 這行代碼根據系統中是否有CUDA可用來選擇設備。如果CUDA可用，設備將被設置為CUDA（GPU），否則設置為CPU。

5. `NUM_EPOCHS = 3`: 設置了訓練迭代的次數，即訓練過程中數據將被遍歷的次數。

總結來說，這段代碼確保了在訓練深度學習模型時，使用固定的隨機種子以及特定的設備（GPU或CPU），從而確保訓練過程的結果是可預測和可重現的。

**Download Dataset**

The following cells will download the IMDB movie review dataset (http://ai.stanford.edu/~amaas/data/sentiment/) for positive-negative sentiment classification in as CSV-formatted file:

In [4]:
url = "https://github.com/rasbt/machine-learning-book/raw/main/ch08/movie_data.csv.gz"
filename = url.split("/")[-1]

with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

with gzip.open('movie_data.csv.gz', 'rb') as f_in:
    with open('movie_data.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

這段程式碼從指定的URL下載壓縮的CSV文件，並將其解壓縮為本地的CSV文件。以下是每行程式碼的詳細解釋：

1. **定義URL和文件名稱：**
   ```python
   url = "https://github.com/rasbt/machine-learning-book/raw/main/ch08/movie_data.csv.gz"
   filename = url.split("/")[-1]
   ```
   - `url`: 指定要下載的文件的URL。
   - `filename`: 從URL中提取的文件名稱。在這個例子中，它會是`movie_data.csv.gz`。

2. **下載壓縮文件：**
   ```python
   with open(filename, "wb") as f:
       r = requests.get(url)
       f.write(r.content)
   ```
   - 使用`requests`庫向指定的URL發送GET請求(`requests.get(url)`)，並將返回的內容寫入本地文件(`f.write(r.content)`)。
   - `with open(filename, "wb") as f:` 打開二進制寫入模式的文件，如果文件不存在，將會被創建。

3. **解壓縮壓縮文件：**
   ```python
   with gzip.open('movie_data.csv.gz', 'rb') as f_in:
       with open('movie_data.csv', 'wb') as f_out:
           shutil.copyfileobj(f_in, f_out)
   ```
   - `gzip.open('movie_data.csv.gz', 'rb')`: 打開壓縮的`movie_data.csv.gz`文件為讀取二進制模式。
   - `open('movie_data.csv', 'wb')`: 打開要寫入解壓縮數據的`movie_data.csv`文件為二進制寫入模式。
   - `shutil.copyfileobj(f_in, f_out)`: 將從`f_in`讀取的數據直接寫入`f_out`，從而將壓縮文件解壓縮為本地的CSV文件。

這樣，你可以從指定的URL下載壓縮的CSV文件並將其解壓縮為本地的可用文件。

Check that the dataset looks okay:

In [5]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


你可以在本地機器上執行這段程式碼，它會將名為`movie_data.csv`的CSV文件讀取到Pandas DataFrame中，然後顯示前幾行數據。這樣你可以查看CSV文件的內容和結構。

In [6]:
df.shape

(50000, 2)

如果你已經成功讀取了`movie_data.csv`文件到Pandas DataFrame中，你可以輕鬆地使用`.shape`屬性來查看DataFrame的形狀（即行數和列數）。

通常情況下，這個屬性會返回一個元組，第一個元素代表行數，第二個元素代表列數。例如，如果你的DataFrame名稱是`df`，那麼可以這樣查看形狀：

```python
df.shape
```

這樣就能顯示出DataFrame的行數和列數。

**Split Dataset into Train/Validation/Test**

In [7]:
train_texts = df.iloc[:35000]['review'].values
train_labels = df.iloc[:35000]['sentiment'].values

valid_texts = df.iloc[35000:40000]['review'].values
valid_labels = df.iloc[35000:40000]['sentiment'].values

test_texts = df.iloc[40000:]['review'].values
test_labels = df.iloc[40000:]['sentiment'].values

這段程式碼從已讀入的CSV檔案中提取了訓練、驗證和測試集的文本資料和情感標籤。

1. **訓練集提取**：
   ```python
   train_texts = df.iloc[:35000]['review'].values
   train_labels = df.iloc[:35000]['sentiment'].values
   ```
   - `train_texts`：從DataFrame `df` 中提取了前35000行的 `review` 列，即電影評論文本的數據，並轉換為NumPy數組。
   - `train_labels`：從DataFrame `df` 中提取了前35000行的 `sentiment` 列，即相應的情感標籤（正面或負面），同樣轉換為NumPy數組。

2. **驗證集提取**：
   ```python
   valid_texts = df.iloc[35000:40000]['review'].values
   valid_labels = df.iloc[35000:40000]['sentiment'].values
   ```
   - `valid_texts`：從DataFrame `df` 中提取了第35000行到第39999行的 `review` 列，作為驗證集的電影評論文本，轉換為NumPy數組。
   - `valid_labels`：從DataFrame `df` 中提取了第35000行到第39999行的 `sentiment` 列，作為驗證集的情感標籤，轉換為NumPy數組。

3. **測試集提取**：
   ```python
   test_texts = df.iloc[40000:]['review'].values
   test_labels = df.iloc[40000:]['sentiment'].values
   ```
   - `test_texts`：從DataFrame `df` 中提取了從第40000行開始至最後一行的 `review` 列，作為測試集的電影評論文本，轉換為NumPy數組。
   - `test_labels`：從DataFrame `df` 中提取了從第40000行開始至最後一行的 `sentiment` 列，作為測試集的情感標籤，轉換為NumPy數組。

這樣的分割方式可以有效地將數據集劃分為訓練、驗證和測試三部分，以便後續進行機器學習模型的訓練、調參和評估。

## Tokenizing the dataset

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

這行代碼是用來從Hugging Face的Transformers庫中加載預訓練的DistilBERT tokenizer。讓我來詳細解釋每一部分的意義：

1. **`from_pretrained('distilbert-base-uncased')`**：
   - `from_pretrained` 是一個Transformer庫中的通用方法，用於從預訓練模型中加載模型或tokenizer。
   - `'distilbert-base-uncased'` 是一個預訓練的DistilBERT模型的名稱。這裡的`'uncased'` 表示模型是通過將所有文本轉換成小寫來訓練的，而 `'base'` 表示這是DistilBERT的基本版本，不含任何添加的層或特殊設置。

2. **`DistilBertTokenizerFast`**：
   - `DistilBertTokenizerFast` 是DistilBERT的一個特殊類，用於快速地進行tokenize和token轉換。這個類能夠處理長文本，並使用多線程進行更快的處理。

3. **`tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')`**：
   - 這行代碼將從Hugging Face模型庫中加載指定名稱的DistilBERT tokenizer。一旦加載完成，`tokenizer` 就會成為一個可以用於tokenize文本的實例。

通常，使用這樣的tokenizer對文本進行tokenize是為了準備輸入資料以供DistilBERT或其他基於Transformer的模型進行處理。

In [9]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

這段程式碼使用了之前加載的`DistilBertTokenizerFast`將訓練集、驗證集和測試集的文本進行了tokenize和編碼。讓我來解釋每行的意義：

1. **`train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)`**：
   - `tokenizer` 是之前從`DistilBertTokenizerFast.from_pretrained`加載的DistilBERT tokenizer的實例。
   - `list(train_texts)` 是訓練集中的所有文本，以列表形式提供給tokenizer。
   - `truncation=True` 表示如果文本長度超過DistilBERT模型的最大輸入長度，將會對文本進行截斷。
   - `padding=True` 表示tokenizer會將所有文本進行填充，確保它們具有相同的長度，以便於批次處理。

2. **`valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)`** 和 **`test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)`**：
   - 這兩行代碼與第一行相似，只是分別將驗證集和測試集的文本列表提供給tokenizer進行tokenize和編碼。

在這些編碼之後，`train_encodings`、`valid_encodings`和`test_encodings`會包含tokenized後的文本，並且已經進行了填充和截斷以準備輸入到DistilBERT或其他基於Transformer的模型中進行訓練或預測。

In [10]:
train_encodings[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

`train_encodings[0]`返回了一個`Encoding`對象，這是由Hugging Face的`tokenizers`庫生成的。這個對象包含了一些重要的屬性，這些屬性描述了經過tokenize和編碼後的文本信息。讓我來詳細解釋一下這些屬性的意義：

- **`ids`**: 這是tokenize後的token IDs列表，即每個token被映射到其對應的ID。

- **`type_ids`**: 在BERT和其變種模型中，這個屬性通常用於區分不同句子的token。在一般的情況下，它會是一個全為0的列表，因為大多數任務都只有一個句子。對於DistilBERT這種單一句子任務，這個屬性的作用不大。

- **`tokens`**: 這是tokenize後的token字符串列表，即原始文本被分成的token。

- **`offsets`**: 這個屬性對應於每個token在原始文本中的起始和結束位置的元組列表。這在需要將模型的輸出映射回原始文本位置時非常有用。

- **`attention_mask`**: 這是一個二進制的遮罩向量，顯示哪些位置是真實的token（1），哪些位置是填充的token（0）。

- **`special_tokens_mask`**: 如果tokenizer使用了特殊token（如`[CLS]`, `[SEP]`），則這個屬性會顯示這些特殊token的位置。

- **`overflowing`**: 如果文本超出了tokenizer的最大token長度（例如BERT的512個token），那麼這個屬性將包含超出部分的附加token。

總結來說，這些屬性組合起來描述了經過DistilBERT tokenizer處理後的文本，使得它們可以被模型處理和理解。

**Dataset Class and Loaders**

In [11]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

這段代碼定義了一個`IMDbDataset`類別，這個類別繼承自PyTorch的`torch.utils.data.Dataset`，用於創建IMDb數據集的數據加載器。讓我來詳細解釋每一行的意義：

1. **`class IMDbDataset(torch.utils.data.Dataset):`**: 定義了一個新的類別`IMDbDataset`，它繼承自`torch.utils.data.Dataset`，這是PyTorch中用於自定義數據集的基類。

2. **`def __init__(self, encodings, labels):`**: 這是類別的初始化方法。它接受兩個參數：
   - `encodings`: 包含了tokenize和編碼後的文本數據，通常是從tokenizer返回的`train_encodings`、`valid_encodings`或`test_encodings`。
   - `labels`: 包含了每個文本對應的標籤數據，例如正面情感或負面情感，通常是`train_labels`、`valid_labels`或`test_labels`。

3. **`self.encodings = encodings`**: 將`encodings`參數保存為類別屬性`encodings`，以便在類別的其他方法中使用。

4. **`self.labels = labels`**: 將`labels`參數保存為類別屬性`labels`，以便在類別的其他方法中使用。

5. **`def __getitem__(self, idx):`**: 這個方法是Dataset類別必須實現的方法之一，用於根據給定的索引`idx`返回數據樣本。在這個類別中：
   - 創建一個名為`item`的字典，用於存儲從`encodings`和`labels`中取出的數據。
   - 使用列表推導式，遍歷`encodings.items()`，並將每個鍵值對的值（即token IDs、attention mask等）轉換為PyTorch的tensor。
   - 將`labels`也轉換為PyTorch的tensor，並放入`item`字典中作為`'labels'`鍵的值。
   - 返回這個`item`字典作為索引`idx`對應的數據樣本。

6. **`def __len__(self):`**: 這個方法同樣是Dataset類別必須實現的方法之一，返回數據集的長度（即樣本數量）。在這個類別中，它返回`labels`的長度，這樣在訓練或測試時，PyTorch的`DataLoader`就知道要迭代多少次了。

7. **`train_dataset = IMDbDataset(train_encodings, train_labels)`**: 通過使用`IMDbDataset`類別來創建`train_dataset`，將訓練集的tokenize和編碼數據`train_encodings`與對應的標籤`train_labels`進行結合。

8. **`valid_dataset = IMDbDataset(valid_encodings, valid_labels)`**: 同樣地，創建`valid_dataset`來存儲驗證集的數據。

9. **`test_dataset = IMDbDataset(test_encodings, test_labels)`**: 最後，創建`test_dataset`來存儲測試集的數據。

這樣，通過`IMDbDataset`類別，你現在擁有了三個準備好的數據集，可以用於訓練、驗證和測試DistilBERT等模型。

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

這段代碼創建了用於訓練、驗證和測試的數據加載器（data loader）。讓我來詳細解釋每一行的意義：

1. **`train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)`**:
   - `torch.utils.data.DataLoader`是PyTorch中用於加載數據的工具類別，它可以將自定義的`Dataset`對象（這裡是`train_dataset`）轉換為可迭代的數據加載器。
   - `train_dataset`是剛剛定義的`IMDbDataset`類別的一個實例，包含了訓練數據的tokenize和編碼數據，以及對應的標籤。
   - `batch_size=16`設置了每個批次的樣本數量為16，這意味著每次從數據加載器中獲取的數據批次將包含16個樣本。
   - `shuffle=True`表示在每個epoch開始前將訓練數據隨機打亂，這有助於模型更好地學習。

2. **`valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)`**:
   - 與`train_loader`類似，創建了用於驗證的數據加載器`valid_loader`。
   - `valid_dataset`是剛剛創建的`IMDbDataset`類別的另一個實例，包含了驗證數據的tokenize和編碼數據，以及對應的標籤。
   - `batch_size=16`與`train_loader`相同，每個批次包含16個樣本。
   - `shuffle=False`表示在驗證期間不需要打亂數據，因為驗證過程中不需要計算梯度和反向傳播，因此可以節省計算資源。

3. **`test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)`**:
   - 與前兩個加載器類似，創建了用於測試的數據加載器`test_loader`。
   - `test_dataset`是剛剛創建的`IMDbDataset`類別的另一個實例，包含了測試數據的tokenize和編碼數據，以及對應的標籤。
   - `batch_size=16`與前兩個相同，每個批次包含16個樣本。
   - `shuffle=False`表示在測試期間不需要打亂數據，以保持數據的原始順序。

這樣，通過`train_loader`、`valid_loader`和`test_loader`這三個數據加載器，你可以方便地在訓練、驗證和測試階段使用DistilBERT等模型。每次迭代時，這些加載器將返回一個包含16個樣本的批次，並且在訓練時還會將數據打亂，以幫助模型更好地學習。

## Loading and fine-tuning a pre-trained BERT model

In [13]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

這段代碼完成了以下操作：

1. **`model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')`**：
   - 使用`from_pretrained`方法從Hugging Face的模型庫中加載了一個預訓練的DistilBERT模型。這個特定模型是基於小型、輕量化版本的DistilBERT，適合文本分類任務。
   - `DistilBertForSequenceClassification`是一個特定於序列分類任務（如情感分析、文本分類等）的DistilBERT模型。它包含了DistilBERT的主體結構以及一個額外的分類層。

2. **`model.to(DEVICE)`**：
   - 將模型移動到指定的設備上，這裡根據可用的硬件（GPU或CPU）選擇。
   - `DEVICE`是在之前定義的，根據系統中是否有可用的GPU，可能是`cuda`（GPU加速）或`cpu`。

3. **`model.train()`**：
   - 設置模型處於訓練模式。這主要是為了啟用模型中的Dropout層和Batch Normalization層等訓練時特定的行為。
   - 在使用`model.train()`後，使用`model.eval()`可以將模型切換回評估模式。

4. **`optim = torch.optim.Adam(model.parameters(), lr=5e-5)`**：
   - 定義了優化器，這裡使用了Adam優化器（一種常用的隨機梯度下降算法的變種）。
   - `model.parameters()`返回模型中所有可學習的參數，這些參數將被優化器用來更新模型的權重。
   - `lr=5e-5`設置了學習率，即每次更新參數時的步長大小。這個值通常需要調整來平衡訓練速度和模型性能。

總結來說，這段代碼初始化了一個用於序列分類的DistilBERT模型，將其移動到適當的設備（GPU或CPU），設置為訓練模式並創建了一個Adam優化器來最小化訓練過程中的損失。接下來，你可以使用`train_loader`來進行模型的訓練。

這段警告信息表明，在初始化`DistilBertForSequenceClassification`時，有一些模型權重並未從`distilbert-base-uncased`模型的檢查點中使用。具體來說：

1. **未使用的權重：**
   - ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
   
   這些權重通常與BERT或DistilBERT模型的語言模型頭部（如預訓練或掩碼語言建模任務）相關，而不是用於序列分類任務。

2. **新初始化的權重：**
   - ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
   
   這些權重是新初始化的，它們是`DistilBertForSequenceClassification`模型特有的分類頭部的權重。這些權重與預訓練模型中的部分權重不同，因為預訓練模型通常不包括這些分類頭部。

3. **警告的含義：**
   - 如果你期望從一個與目標任務相同或非常相似的模型檢查點初始化模型，這個警告是正常的。這種情況下，你應該確保模型的預訓練任務和目標任務相符合。
   - 如果你期望使用完全相同的模型來初始化，這個警告表明模型檢查點可能不符合預期，你應該確保使用正確的檢查點或進行適當的微調來將模型訓練用於具體的下游任務。

總結來說，這個警告提醒你確保模型的初始化與預期任務相符，並建議進行適當的訓練來使模型適應具體的下游任務，以便進行預測和推理操作。

**Train Model -- Manual Training Loop**

In [14]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0
        
        for batch_idx, batch in enumerate(data_loader):
        
        ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()
        
        return correct_pred.float()/num_examples * 100


這段代碼是用來計算模型在給定數據集上的準確率的函數。讓我們來逐行解釋其含義：

```python
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():  # 確保在評估模式下不進行梯度計算
        correct_pred, num_examples = 0, 0
        
        for batch_idx, batch in enumerate(data_loader):
        
            # 將數據移動到指定的設備（GPU或CPU）
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # 通過模型獲取預測結果
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']  # 獲取模型的預測 logits（未經過 softmax 的輸出）
            
            # 將 logits 轉換為預測的類別標籤
            predicted_labels = torch.argmax(logits, 1)
            
            # 計算準確的預測數量
            num_examples += labels.size(0)  # 累計樣本數量
            correct_pred += (predicted_labels == labels).sum()  # 累計正確預測的數量
        
        # 返回準確率（以百分比形式）
        return correct_pred.float() / num_examples * 100
```

這個函數主要用於以下幾個步驟：

1. **`with torch.no_grad():`**：
   - 這個語句確保在執行函數內部時，PyTorch不會跟蹤和計算梯度，這樣可以節省內存和加速計算。

2. **迴圈`for batch_idx, batch in enumerate(data_loader):`**：
   - 這個迴圈遍歷了提供的`data_loader`中的每個批次數據。

3. **數據準備**：
   - `input_ids`、`attention_mask`和`labels`是從每個批次中的字典`batch`中提取的，這些鍵名稱與`IMDbDataset`類中的`__getitem__`方法返回的項目對應。
   - 這些數據通過`to(device)`方法移動到指定的設備（GPU或CPU）上進行計算。

4. **模型預測**：
   - 通過`model(input_ids, attention_mask=attention_mask)`調用模型，將`input_ids`和`attention_mask`傳遞給模型進行前向傳播。
   - 從模型輸出中獲取預測的`logits`，這些`logits`是未經過 softmax 處理的輸出結果。

5. **計算準確率**：
   - 使用`torch.argmax(logits, 1)`獲取每個樣本的預測類別。
   - 通過比較`predicted_labels`和`labels`計算正確預測的數量。
   - `correct_pred`變量累計正確預測的數量，`num_examples`變量累計處理的總樣本數量。

6. **返回結果**：
   - 最後計算並返回準確率，即正確預測的百分比。

In [15]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_loader):
        
        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
        
        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nValid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 0001/0003 | Batch 0000/2188 | Loss: 0.6771
Epoch: 0001/0003 | Batch 0250/2188 | Loss: 0.3006
Epoch: 0001/0003 | Batch 0500/2188 | Loss: 0.3678
Epoch: 0001/0003 | Batch 0750/2188 | Loss: 0.1487
Epoch: 0001/0003 | Batch 1000/2188 | Loss: 0.6674
Epoch: 0001/0003 | Batch 1250/2188 | Loss: 0.3264
Epoch: 0001/0003 | Batch 1500/2188 | Loss: 0.4358
Epoch: 0001/0003 | Batch 1750/2188 | Loss: 0.2579
Epoch: 0001/0003 | Batch 2000/2188 | Loss: 0.2474
Training accuracy: 96.32%
Valid accuracy: 92.34%
Time elapsed: 20.67 min
Epoch: 0002/0003 | Batch 0000/2188 | Loss: 0.0850
Epoch: 0002/0003 | Batch 0250/2188 | Loss: 0.3433
Epoch: 0002/0003 | Batch 0500/2188 | Loss: 0.0793
Epoch: 0002/0003 | Batch 0750/2188 | Loss: 0.0061
Epoch: 0002/0003 | Batch 1000/2188 | Loss: 0.1536
Epoch: 0002/0003 | Batch 1250/2188 | Loss: 0.0816
Epoch: 0002/0003 | Batch 1500/2188 | Loss: 0.0786
Epoch: 0002/0003 | Batch 1750/2188 | Loss: 0.1395
Epoch: 0002/0003 | Batch 2000/2188 | Loss: 0.0344
Training accuracy: 98.35%
V

這段代碼是訓練和評估DistilBERT模型的訓練過程，讓我們逐步解釋：

1. **開始時間記錄**：
   ```python
   start_time = time.time()
   ```
   這行代碼記錄了訓練開始的時間，用於後續計算整個訓練過程的耗時。

2. **訓練迴圈**：
   ```python
   for epoch in range(NUM_EPOCHS):
   ```
   這是訓練的主迴圈，迭代指定次數的訓練輪次（epochs）。

3. **進入訓練模式**：
   ```python
   model.train()
   ```
   這行將模型設置為訓練模式，這主要是為了啟用Dropout和Batch Normalization等訓練模式下的特定操作。

4. **遍歷數據加載器**：
   ```python
   for batch_idx, batch in enumerate(train_loader):
   ```
   在每個訓練輪次中，遍歷訓練數據加載器中的每個批次數據。

5. **準備數據**：
   ```python
   input_ids = batch['input_ids'].to(DEVICE)
   attention_mask = batch['attention_mask'].to(DEVICE)
   labels = batch['labels'].to(DEVICE)
   ```
   將從數據加載器中獲取的數據轉移到指定的設備（GPU或CPU）上進行計算。

6. **前向傳播**：
   ```python
   outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
   loss, logits = outputs['loss'], outputs['logits']
   ```
   通過模型獲取輸出，並從輸出中獲取損失（loss）和預測的 logits。

7. **反向傳播和優化**：
   ```python
   optim.zero_grad()
   loss.backward()
   optim.step()
   ```
   這部分代碼執行反向傳播，計算梯度並更新模型的參數。

8. **日誌記錄**：
   ```python
   if not batch_idx % 250:
       print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
              f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
              f'Loss: {loss:.4f}')
   ```
   每250個批次打印一次當前的訓練進度（輪次、批次、損失值）。

9. **進入評估模式**：
   ```python
   model.eval()
   ```
   訓練過程結束後，將模型設置為評估模式，主要是為了在評估過程中禁用Dropout等操作。

10. **評估模型**：
    ```python
    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nValid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
    ```
    使用`compute_accuracy`函數計算並打印訓練集和驗證集的準確率。

11. **打印耗時**：
    ```python
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    ```
    每輪訓練結束後打印耗時時間。

12. **總訓練時間**：
    ```python
    print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
    ```
    打印整個訓練過程的總耗時時間。

13. **測試集準確率**：
    ```python
    print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')
    ```
    使用`compute_accuracy`函數計算並打印測試集的準確率。

這段代碼完整地實現了訓練過程，包括設置模型、優化器、訓練和評估模式的切換，以及準確率的計算和日誌的打印。

In [16]:
del model # free memory

這行程式碼 `del model` 是在Python中用來刪除物件的語句。在這裡，它的作用是刪除名為 `model` 的變數，這樣可以釋放佔用的記憶體空間，讓系統可以在後續的運算中再次使用這部分記憶體。這對於需要釋放模型佔用的大量記憶體資源時很有用，特別是在處理大型模型或需要長時間執行的程式時。

### Fine-tuning a transformer more conveniently using the Trainer API

Reload pretrained model:

In [17]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train();

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

這段程式碼是用來重新載入並準備一個來自 `distilbert-base-uncased` 預訓練模型的 `DistilBertForSequenceClassification` 實例。讓我們一步步來解釋：

1. **`DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')`**：
   - 這個函數是從 Hugging Face Transformers 模型庫中載入一個預訓練的 DistilBERT 模型，並初始化為一個序列分類任務的模型。`'distilbert-base-uncased'` 是模型的名稱，表示這是一個小型的基於 DistilBERT 的預訓練模型，支援小寫字母和未標記的分詞。

2. **`model.to(DEVICE)`**：
   - 將模型移動到指定的計算裝置，這裡的 `DEVICE` 可能是 GPU 或 CPU，取決於系統中可用的硬體資源。通常，如果有 GPU 可用，模型會移動到 GPU 上以加速訓練和推理。

3. **`model.train()`**：
   - 將模型設置為訓練模式。這告訴模型在訓練過程中要計算梯度，以便後續調用優化器進行參數更新。

總結來說，這段程式碼是準備一個可以用於序列分類任務的 DistilBERT 模型，並將其設置為訓練模式，同時移動到適當的計算裝置上。這樣可以開始訓練模型或進行後續的預測和評估。

這段訊息是來自於 Hugging Face Transformers 套件，告知你在初始化 `DistilBertForSequenceClassification` 模型時發生了一些權重未使用和新初始化的情況。讓我們來解釋一下每個部分的含義：

1. **未使用的權重**：
   - 這些權重來自於預訓練的 `distilbert-base-uncased` 模型，但在初始化 `DistilBertForSequenceClassification` 時未被使用。具體來說，包括了：
     - `vocab_projector.bias`
     - `vocab_transform.weight`
     - `vocab_projector.weight`
     - `vocab_transform.bias`
     - `vocab_layer_norm.bias`
     - `vocab_layer_norm.weight`
   - 這是預期的情況，因為 `DistilBertForSequenceClassification` 可能使用了與預訓練模型不同的架構或在不同的任務上進行了微調。

2. **新初始化的權重**：
   - 這些權重是在 `DistilBertForSequenceClassification` 初始化過程中新建的，因為它們在預訓練模型中不存在或不直接相關。具體包括：
     - `pre_classifier.weight`
     - `classifier.weight`
     - `classifier.bias`
     - `pre_classifier.bias`
   - 這些權重是專為序列分類任務而初始化的，通常包括最後一層分類器和與之相關的權重。

總結來說，這段訊息提醒你，初始化的模型中有一些權重來自於預訓練模型，但並非所有權重都被重複使用，部分可能是因為模型架構或任務不同所導致的。這是正常的行為，只要確保將模型訓練在你的特定任務上，以便使用它進行預測和推論。

In [18]:
from transformers import Trainer, TrainingArguments


optim = torch.optim.Adam(model.parameters(), lr=5e-5)
training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=3,     
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

在這段程式碼中，你使用了 Hugging Face Transformers 套件中的 `Trainer` 和 `TrainingArguments` 來進行模型訓練。讓我來解釋一下每行代碼的作用：

1. **導入套件**：
   ```python
   from transformers import Trainer, TrainingArguments
   ```
   - 導入了 `Trainer` 和 `TrainingArguments` 類別，它們分別用於訓練和訓練參數的設定。

2. **定義優化器**：
   ```python
   optim = torch.optim.Adam(model.parameters(), lr=5e-5)
   ```
   - 定義了一個 Adam 優化器，用於訓練模型。這裡將模型的參數和學習率（lr=5e-5）作為優化器的參數。

3. **設置訓練參數**：
   ```python
   training_args = TrainingArguments(
       output_dir='./results', 
       num_train_epochs=3,     
       per_device_train_batch_size=16, 
       per_device_eval_batch_size=16,   
       logging_dir='./logs',
       logging_steps=10,
   )
   ```
   - 創建了一個 `TrainingArguments` 物件 `training_args`，用來設置訓練的相關參數。具體參數設置包括：
     - `output_dir`：訓練結果和模型儲存的目錄。
     - `num_train_epochs`：訓練的總epoch數。
     - `per_device_train_batch_size`：每個訓練設備（GPU或CPU）的批次大小。
     - `per_device_eval_batch_size`：每個評估設備（GPU或CPU）的批次大小。
     - `logging_dir`：日誌文件儲存的目錄。
     - `logging_steps`：每隔多少步寫入一次日誌。

4. **創建 `Trainer` 實例**：
   ```python
   trainer = Trainer(
       model=model,
       args=training_args,
       train_dataset=train_dataset,
   )
   ```
   - 使用創建的 `model` 和 `training_args` 來實例化 `Trainer` 物件 `trainer`。`Trainer` 將管理模型訓練的所有過程，包括前向傳播、反向傳播、優化器更新等。

這段程式碼設置了一個基本的訓練流程，使用了 `Trainer` 和 `TrainingArguments` 來方便地進行模型訓練，同時管理訓練參數和日誌記錄。接下來，你可以使用 `trainer.train()` 方法來開始訓練模型。

In [19]:
# install dataset via pip install datasets
from datasets import load_metric
import numpy as np


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred # logits are a numpy array, not pytorch tensor
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
               predictions=predictions, references=labels)

這段程式碼主要用於設定並使用 `datasets` 函式庫來加載評估指標（accuracy），並定義了一個計算評估指標的函式 `compute_metrics`。讓我們逐行來解釋每段程式碼的意義：

1. **安裝 `datasets` 函式庫**：
   ```python
   # install dataset via pip install datasets
   ```
   - 這是一個註解，指示你可以通過使用 `pip install datasets` 來安裝 `datasets` 函式庫，它提供了訪問各種常用於機器學習的數據集和指標的介面。

2. **導入必要的庫**：
   ```python
   from datasets import load_metric
   import numpy as np
   ```
   - 導入 `datasets` 函式庫中的 `load_metric` 函式，用於加載特定的指標（在這裡是 `accuracy`）。
   - 導入 `numpy` 庫並縮寫為 `np`，這將用於數組操作。

3. **加載指標**：
   ```python
   metric = load_metric("accuracy")
   ```
   - 使用 `load_metric` 函式從 `datasets` 函式庫中加載 `accuracy` 指標。這個指標將用於在模型評估期間計算評估指標。

4. **定義 `compute_metrics` 函式**：
   ```python
   def compute_metrics(eval_pred):
       logits, labels = eval_pred  # logits 是一個 numpy 數組，不是 pytorch 張量
       predictions = np.argmax(logits, axis=-1)
       return metric.compute(predictions=predictions, references=labels)
   ```
   - `compute_metrics` 函式接受一個 `eval_pred` 參數，其中包含模型預測的 logits（對應於類別的分數）和真實的標籤 `labels`。
   - 使用 `numpy` 的 `argmax` 函式來計算每個樣本預測的類別索引，`axis=-1` 表示在最後一個維度上執行求取最大值的操作。
   - 最後，調用 `metric.compute` 函式計算並返回預測結果 `predictions` 與真實標籤 `labels` 之間的評估指標（在這裡是準確度）。

In [20]:
optim = torch.optim.Adam(model.parameters(), lr=5e-5)


training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=3,     
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    optimizers=(optim, None) # optimizer and learning rate scheduler
)

# force model to only use 1 GPU (even if multiple are availabe)
# to compare more fairly to previous code

trainer.args._n_gpu = 1

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


這段程式碼是在設置與使用 `Trainer` 物件來進行模型的訓練和評估。讓我們來逐行解釋每段程式碼的意義：

1. **設置優化器**：
   ```python
   optim = torch.optim.Adam(model.parameters(), lr=5e-5)
   ```
   - 創建了一個 Adam 優化器來優化模型參數，學習率設置為 `5e-5`。

2. **設置訓練參數**：
   ```python
   training_args = TrainingArguments(
       output_dir='./results', 
       num_train_epochs=3,     
       per_device_train_batch_size=16, 
       per_device_eval_batch_size=16,   
       logging_dir='./logs',
       logging_steps=10
   )
   ```
   - 創建了一個 `TrainingArguments` 對象，這裡設置了模型訓練的各種參數，包括：
     - `output_dir`: 模型訓練過程中輸出的目錄。
     - `num_train_epochs`: 訓練的總 epoch 數。
     - `per_device_train_batch_size`: 每個設備上的訓練批次大小。
     - `per_device_eval_batch_size`: 每個設備上的評估批次大小。
     - `logging_dir`: 訓練過程中日誌輸出的目錄。
     - `logging_steps`: 每隔多少步驟記錄一次訓練日誌。

3. **創建 `Trainer` 物件**：
   ```python
   trainer = Trainer(
       model=model,
       compute_metrics=compute_metrics,
       args=training_args,
       train_dataset=train_dataset,
       eval_dataset=test_dataset,
       optimizers=(optim, None) # optimizer and learning rate scheduler
   )
   ```
   - 創建了一個 `Trainer` 物件，用於訓練和評估模型。
   - `model`: 設置要訓練的模型。
   - `compute_metrics`: 設置用於計算評估指標的函式。
   - `args`: 設置訓練的參數，這裡使用了之前定義的 `training_args`。
   - `train_dataset`: 設置訓練所使用的數據集。
   - `eval_dataset`: 設置評估所使用的數據集。
   - `optimizers`: 設置優化器和學習率調度器，這裡只設置了優化器為之前定義的 `optim`。

4. **強制使用單 GPU**：
   ```python
   trainer.args._n_gpu = 1
   ```
   - 強制設置 `Trainer` 物件只使用一個 GPU 進行訓練，即使有多個 GPU 可用，這樣可以更公平地與之前的程式碼進行比較。

這樣的設置使得可以通過 `Trainer` 物件更方便地管理模型訓練的過程，包括了記錄日誌、處理不同的設備、管理優化器等功能，使得整個訓練過程更加方便和高效。

這個警告訊息是關於未來版本中 PyTorch 相關訓練參數 `--report_to` 默認值的更改。目前的警告建議在未來的 v5 版本中，默認值將從當前的所有安裝的集成更改為 `none`。這意味著在未來版本中，如果需要保留相同的行為，就需要明確地設置 `--report_to all`。

根據警告信息，建議開始更新程式碼，並確保在將來的版本中能夠正確處理這些變更，以避免潛在的影響或錯誤。

In [21]:
start_time = time.time()
trainer.train()
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

***** Running training *****
  Num examples = 35000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6564


Step,Training Loss
10,0.705800
20,0.684100
30,0.681500
40,0.591600
50,0.328600
60,0.478300
70,0.426300
80,0.356900
90,0.359900
100,0.268200


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

Total Training Time: 45.36 min


這段程式碼是使用 `Trainer` 物件來進行模型訓練的準備與設置。讓我們來詳細解釋每一行的意義：

1. **start_time = time.time()**: 記錄當前時間，用於後續計算訓練所花費的時間。

2. **trainer.train()**: 啟動訓練過程。這裡的 `trainer` 是之前設置好的 `Trainer` 物件，它包含了模型、訓練參數以及訓練數據集等設定。

3. **print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')**: 計算並輸出整個訓練過程所花費的時間，以分鐘為單位。`time.time() - start_time` 計算從 `start_time` 到當前的時間差，除以 60 得到分鐘數，`.2f` 表示保留兩位小數。

這段程式碼的目的是訓練 `Trainer` 物件中指定的模型，並監控訓練時間。通過 `Trainer` 對象，可以更容易地管理和設置訓練過程中的各種參數和資源。

這段訓練過程的日誌信息顯示了訓練的基本設置和參數，讓我們逐一解釋：

- **Num examples = 35000**: 訓練數據集中的樣本數量，這裡是35000個樣本。

- **Num Epochs = 3**: 訓練過程中的 epoch 數量，即完整遍歷訓練數據集的次數。

- **Instantaneous batch size per device = 16**: 每個訓練設備（如 GPU）的即時批次大小，即每次訓練時，每個設備處理的樣本數量。

- **Total train batch size (w. parallel, distributed & accumulation) = 16**: 總的訓練批次大小，考慮到平行處理、分佈式訓練和梯度累積等因素後的批次大小。

- **Gradient Accumulation steps = 1**: 梯度累積步驟，即在執行反向傳播之前累積梯度的步數。這裡設置為1，表示每個批次後立即執行反向傳播更新模型參數。

- **Total optimization steps = 6564**: 總的優化步驟數，即總共需要更新模型參數的次數。計算方式是 `Num examples * Num Epochs / Total train batch size`，這裡的值是根據上述參數計算出來的。

In [22]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


{'eval_loss': 0.30534815788269043,
 'eval_accuracy': 0.9327,
 'eval_runtime': 87.1161,
 'eval_samples_per_second': 114.789,
 'eval_steps_per_second': 7.174,
 'epoch': 3.0}

`trainer.evaluate()` 的功能是用來評估模型在驗證集或測試集上的性能表現。根據之前的設置，`eval_dataset` 參數應該是設置為測試集 `test_dataset`，因此 `trainer.evaluate()` 會計算模型在測試集上的指標，如準確率、損失等，具體取決於在 `compute_metrics` 函數中定義的評估指標。

看起來你正在觀察訓練過程中的進度更新。這個更新顯示了模型訓練過程中的批次進度，其中 `625/625` 表示目前訓練到的批次數量，`01:26` 則表示已經過去的時間（小時:分鐘）。這些數據幫助你了解訓練的進度和時間消耗。

這些指標是在模型評估（evaluate）過程中計算得到的結果。讓我們一一解釋這些指標的含義：

- `eval_loss`: 在評估過程中計算得到的平均損失值。這表示模型在測試集上的預測與實際標籤之間的差異程度。
  
- `eval_accuracy`: 在評估過程中計算得到的準確率。這表示模型在測試集上正確預測的比例，通常以百分比表示。

- `eval_runtime`: 評估過程的執行時間，單位是秒。

- `eval_samples_per_second`: 評估過程中每秒處理的樣本數。

- `eval_steps_per_second`: 評估過程中每秒完成的步驟數。

- `epoch`: 目前的訓練周期數。

In [23]:
model.eval()
model.to(DEVICE)
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Test accuracy: 93.27%


在這段程式碼中，我們將模型設置為評估模式（eval mode），然後將其移動到適當的裝置（DEVICE）。接著，我們使用之前定義的 `compute_accuracy` 函式來計算模型在測試集上的準確率，並將結果以百分比形式印出。

這段程式碼的流程如下：
1. `model.eval()`: 將模型設置為評估模式。在這個模式下，模型的行為與訓練模式不同，特別是在評估過程中不會計算梯度或者進行反向傳播。
   
2. `model.to(DEVICE)`: 將模型移動到指定的裝置（這裡是 CUDA 裝置，如果可用的話）。

3. `compute_accuracy(model, test_loader, DEVICE)`: 調用 `compute_accuracy` 函式計算模型在測試集上的準確率。

4. `print(f'Test accuracy: {accuracy:.2f}%')`: 將計算得到的測試準確率印出，並格式化成百分比形式。

測試準確率為 93.27%。這表示在測試集上，模型對情感分類任務的準確率為93.27%。這是一個很好的結果，顯示了模型在這個任務上的良好表現。

...

---

Readers may ignore the next cell.

In [4]:
! python ../.convert_notebook_to_script.py --input ch16-part3-bert.ipynb --output ch16-part3-bert.py

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook ch16-part3-bert.ipynb to script
[NbConvertApp] Writing 9089 bytes to ch16-part3-bert.py
